This describes the Random Forest analysis. 
outcome: gname, 63 categories, each having at least 150 entries
features: 35, listed in final_data_list.xlsx
rows (entries): 38251 total entries

In [ ]:
# Y_in.csv had the outcome as 1 column with 63 categories. For Random Forest
# the outcomes were recoded as 63 columns and put into Y_out.xlsx

import math
import numpy as np
import re
import string
import scipy
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import datetime as dt

home = r"C:\Users\ibshi\Desktop\startup.ml\challenge 2\global terrorism\data"
infile = home + r"\Y_in.csv"
Y_in = pd.read_csv(infile,header=None)

temp = Y_in[0].value_counts()
terrlist = list(temp.index)
nrows = len(Y_in.index)
ncols = len(terrlist)
Y_out = pd.DataFrame(np.zeros((nrows,ncols)),columns=terrlist)
Y_temp=pd.concat([Y_in,Y_out],axis=1)

for terr in terrlist:
    Y_temp[terr][Y_temp[0]==terr]=1

Y_temp.drop('0',inplace=True,axis=1)
Y_out = Y_temp

outfile = home + r"\Y_out.xlsx"
Y_out.to_excel(outfile)


In [ ]:
# This runs Random Forest on the full model, default settings
# the data were divided in half for the training and test datasets

# the results from the full model
# accuracy test = 0.813091441
# accuracy train = 0.983057938
# The full results are in <results random forest.xlsx>.
# This result is the first line of <results random forest.xlsx>.

import math
import numpy as np
import re
import string
import scipy
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import datetime as dt

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer

#home = r"C:\Users\ibshi\Desktop\startup.ml\challenge 2\global terrorism\data"
infile = home + r"\final_data.xlsx"
final_data = pd.read_excel(infile)

infile = home + r"\Y_out.xlsx"
Y = pd.read_excel(infile)

# You have to make the categorical variables into a series of dummy variables. 

catlist = ['country',
'region',
'attacktype1',
'attacktype2',
'targtype1',
'targsubtype1',
'claimmode',
'weaptype1',
'WeapRecode1',
'weaptype2',
'WeapRecode2',
'hostkidoutcome']

for cat in catlist:
    hold = pd.get_dummies(final_data[cat])
    final_data = pd.concat([final_data, hold], axis=1)

final_data.drop(catlist,inplace=True,axis=1)

#impute NaN values (replace with mean or median)

imp = Imputer(missing_values='NaN', strategy='mean', axis=1)
imputed_data = pd.DataFrame(imp.fit_transform(final_data))

# create training and test datasets, 0.5 training and 0.5 test

prop_train = 0.5
numtrain = int(prop_train* len(Y))

# randomize whether entry is in test or training

sorter = pd.DataFrame(data=np.zeros(len(Y)),columns=['sorter'])
sorter.sorter[0:numtrain-1] = 1
np.random.shuffle(sorter.sorter)

t_data = pd.concat([sorter,imputed_data],axis=1)
t_Y = pd.concat([sorter,Y],axis=1)

train = t_data[t_data.sorter == 1]
trainY = t_Y[t_Y.sorter == 1]

test = t_data[t_data.sorter == 0]
testY = t_Y[t_Y.sorter == 0]

train.drop(['sorter'],inplace=True,axis=1)
trainY.drop(['sorter'],inplace=True,axis=1)
test.drop(['sorter'],inplace=True,axis=1)
testY.drop(['sorter'],inplace=True,axis=1)

# Run Random Forest

RF = sklearn.ensemble.RandomForestClassifier()

RF.fit(train,trainY)
out = RF.predict(test)
outDF = pd.DataFrame(out)
sc_test = RF.score(test,testY)
print "accuracy on test set"
print sc_test
sc_train = RF.score(train,trainY)
print "accuracy on train set"
print sc_train


In [ ]:
# This runs the Random Forest with the ability to drop features from the
# analysis (in the droplist). This code was run to tune the RF model,
# and the results can be found in <results random forest.xlsx>. Changing
# the number of estimators did not seem to have an effect.  Dropping
# certain features somewhat improved the model performance. This
# tuning resulted in the model found below, and summarized below:

# features in best model:
# iyear
# iday
# Weekday (added)
# country
# region
# nkill
# nhostkid
# ransom
# INT_LOG
# INT_IDEO

# The best model was run 20 times, with the following results
# can be found in <results random forest.xlsx>.
# mean accuracy, test = 0.86838
# sd accuracy, test   = 0.00261
# mean accuracy, training = 0.98068
# sd accuracy, training   = 0.00112

import math
import numpy as np
import re
import string
import scipy
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import datetime as dt

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer

home = r"C:\Users\ibshi\Desktop\startup.ml\challenge 2\global terrorism\data"
infile = home + r"\final_data.xlsx"
final_data = pd.read_excel(infile)

infile = home + r"\Y_out.xlsx"
Y = pd.read_excel(infile)

# determine which feature to be dropped

droplist = ['weaptype1','weaptype2','WeapRecode1','WeapRecode2','attacktype2',
'hostkidoutcome','claimmode','targsubtype1','targtype1','attacktype1',
'imonth','crit1','crit2','crit3','multiple','success','suicide','nperps',
'nperpcap','claimed','nwound','propextent','ndays','nreleased','INT_MISC']

final_data.drop(droplist,inplace=True,axis=1)

# You have to make the categorical variables into a series of dummy variables. 

catlist = ['country',
'region']

for cat in catlist:
    hold = pd.get_dummies(final_data[cat])
    final_data = pd.concat([final_data, hold], axis=1)

final_data.drop(catlist,inplace=True,axis=1)

#impute NaN values (replace with mean or median)

imp = Imputer(missing_values='NaN', strategy='mean', axis=1)
imputed_data = pd.DataFrame(imp.fit_transform(final_data))

# randomize whether entry is in test or training

prop_train = 0.5
numtrain = int(prop_train* len(Y))

sorter = pd.DataFrame(data=np.zeros(len(Y)),columns=['sorter'])
sorter.sorter[0:numtrain-1] = 1
np.random.shuffle(sorter.sorter)

t_data = pd.concat([sorter,imputed_data],axis=1)
t_Y = pd.concat([sorter,Y],axis=1)

train = t_data[t_data.sorter == 1]
trainY = t_Y[t_Y.sorter == 1]

test = t_data[t_data.sorter == 0]
testY = t_Y[t_Y.sorter == 0]

train.drop(['sorter'],inplace=True,axis=1)
trainY.drop(['sorter'],inplace=True,axis=1)
test.drop(['sorter'],inplace=True,axis=1)
testY.drop(['sorter'],inplace=True,axis=1)

# Run Random Forest

RF = sklearn.ensemble.RandomForestClassifier()

RF.fit(train,trainY)
out = RF.predict(test)
outDF = pd.DataFrame(out)
sc_test = RF.score(test,testY)
print "accuracy on test set"
print sc_test
sc_train = RF.score(train,trainY)
print "accuracy on train set"
print sc_train



In [ ]:
# In this section changes to the model are considered

# 1. One aspect is that I used a limited set of available features.  
#    Adding more features might have improved performance.  However,
#    as the the model either improved or remained the same when
#    generally removing features from the original set of 35, it 
#    doesn't seem likely that this would lead to substantial improvement. 

# 2. Looking over the dropped features, it seems that 'city' might be
#    a useful feature to add, given the use of 'country' and 'region'
#    in the final model. 

home = r"C:\Users\ibshi\Desktop\startup.ml\challenge 2\global terrorism\data"
infile = home + r"\globalterrorismdb_0616dist.xlsx"
indata = pd.read_excel(infile)

inseries = indata.city[pd.notnull(indata.city)]
unique_city = list(np.unique(inseries))
print len(unique_city)

#    However, a check of the number of unique cities in the dataset
#    gives 31324, which seems too large to be useful.

# 3. There were a number of unstructured text fields: 'summary','motive',
#    'ransomnote', and 'addnotes'

#    The code below creates lists of unique words for each of these
#    fields.  There are too many unique words for these to be used 
#    unprocessed. However, perhaps some processing (i.e., for certain
#    key words, or by language) might lead to useful new features.

import math
import numpy as np
import re
import string
import scipy
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import datetime as dt

def find_unique(t):
    holdlist =list('a')
    for i in list(t.index):
        temp = t[i]
        temp = temp.lower()
        templist = re.findall('[a-z]+',temp)
        holdlist = holdlist + templist   
    unique_list = list(np.unique(holdlist))
    return unique_list
    
home = r"C:\Users\ibshi\Desktop\startup.ml\challenge 2\global terrorism\data"
infile = home + r"\globalterrorismdb_0616dist.xlsx"
indata = pd.read_excel(infile)

inSeries = indata.motive[pd.notnull(indata.motive)]
outlist = find_unique(inSeries)
outfile = home + r"\unique_motive_list.csv"
np.savetxt(outfile,outlist,delimiter=",", fmt='%s')
#print len(outlist)

inSeries = indata.summary[pd.notnull(indata.summary)]
outlist = find_unique(inSeries)
outfile = home + r"\unique_summary_list.csv"
np.savetxt(outfile,outlist,delimiter=",", fmt='%s')

inSeries = indata.ransomnote[pd.notnull(indata.ransomnote)]
outlist = find_unique(inSeries)
outfile = home + r"\unique_ransomnote_list.csv"
np.savetxt(outfile,outlist,delimiter=",", fmt='%s')

inSeries = indata.addnotes[pd.notnull(indata.addnotes)]
outlist = find_unique(inSeries)
outfile = home + r"\unique_addnotes_list.csv"
np.savetxt(outfile,outlist,delimiter=",", fmt='%s')

